In [ ]:
from pydub import AudioSegment 
import time
from fastcore.all import *
from datetime import datetime
from pydub import AudioSegment


In [ ]:
def str2path(file):
    return file if isinstance(file, Path) else Path(file)

In [ ]:
def time2msec(time:str):
    pt = datetime.strptime(time,'%H:%M:%S' )
    return (pt.hour * 3600 + pt.minute * 60 + pt.second) * 1000

In [ ]:
def loadtsfile(file):
    file = str2path(file)
    with open(file,'r') as infile:
        ts = infile.readlines()
    tss = [x.strip('\n').split('\t')[0:3] for x in ts if len(x.split('\t'))>1]
    return [{'speaker':x[0], 'start':x[1], 'end':x[2]} for x in tss]


In [ ]:
def split_audio(
    input_file:Union[str,Path],
    ts_file:list[dict],
    output_folder:Union[str,Path] = None
):
    input_file = str2path(input_file)
    ts_file = str2path(ts_file)
    output_folder = str2path(output_folder) if output_folder else input_file.parent/input_file.stem
    
    if not output_folder.exists():
        output_folder.mkdir()

    
    ts_d = loadtsfile(ts_file)
    
    audio = AudioSegment.from_file(input_file)
    for i,limit in enumerate(ts_d):
        if limit['start'] == '0':
            limit['start'] = '0:0:0'
        clip = audio[time2msec(limit['start']):time2msec(limit['end'])]
        clip.export(output_folder/f"{limit['speaker']}_{i}.mp3", format='mp3')

In [ ]:
split_audio('Audios/Trump/Trump_2.mp3', 'Audios/Trump/ts_trump_2.txt')

In [ ]:
p = Path('Audios/Trump/')

In [ ]:
for path in p.ls():
    if path.is_dir():
        print(path)

Audios/Trump/Trump_2
Audios/Trump/Trump_1
Audios/Trump/Trump_0


In [ ]:
MP3_FILES = Path('Audios/Trump/Trump_2').rglob('**/*.mp3')
MP3_FILES = [x for x in MP3_FILES]


In [ ]:
silence = AudioSegment.silent(duration=2000)

In [ ]:
MP3_FILES[0].parent/f'{MP3_FILES[0].stem}.mp3'

Path('Audios/Trump/Trump_2/SPEAKER 1_84.mp3')

In [ ]:
combined = AudioSegment.empty()
for i, mp3 in enumerate(MP3_FILES):
    combined += AudioSegment.from_file(mp3.as_posix())
    combined += silence


In [ ]:
combined.export('./Trump_2.mp3', format='mp3')

<_io.BufferedRandom name='./Trump_2.mp3'>

In [ ]:
def merge_mp3s(
    parent_path:Union[str, Path],
    maxlength:int = 9
):
    p = str2path(parent_path)
    silence = AudioSegment.silent(duration=1500)
    for path in p.ls():
        if path.is_dir():
            combined = AudioSegment.empty()            
            count = 0
            MP3_FILES = path.rglob('**/*.mp3')
            MP3_FILES = list(MP3_FILES)
            for i, mp3 in enumerate(MP3_FILES):
                try:
                    combined += AudioSegment.from_file(mp3.as_posix())
                    combined += silence
                except:
                    continue
                count += 1
                if count >= maxlength:
                    combined.export(path/f"{path.stem}_{i}.mp3", format='mp3')
                    combined = AudioSegment.empty()   
                    count = 0
                    print(f"batch {i} {path} Merged")
                elif i+1 == len(MP3_FILES):
                    combined.export(path/f"{path.stem}_{i}.mp3", format='mp3')
            _ = [x.unlink() for x in MP3_FILES]
            print(f"Directory {path} Merged")
            

In [ ]:
merge_mp3s('Audios/Neymar/')

batch 8 Audios/Neymar/Neymar_2 Merged
Directory Audios/Neymar/Neymar_2 Merged
batch 8 Audios/Neymar/Neymar_1 Merged
batch 17 Audios/Neymar/Neymar_1 Merged
batch 26 Audios/Neymar/Neymar_1 Merged
batch 35 Audios/Neymar/Neymar_1 Merged
Directory Audios/Neymar/Neymar_1 Merged
batch 8 Audios/Neymar/Neymar_0 Merged
batch 17 Audios/Neymar/Neymar_0 Merged
batch 26 Audios/Neymar/Neymar_0 Merged
batch 35 Audios/Neymar/Neymar_0 Merged
batch 44 Audios/Neymar/Neymar_0 Merged
batch 53 Audios/Neymar/Neymar_0 Merged
batch 62 Audios/Neymar/Neymar_0 Merged
batch 71 Audios/Neymar/Neymar_0 Merged
Directory Audios/Neymar/Neymar_0 Merged


In [ ]:
with open('./Audios/Biden/ts_biden1.txt','r') as infile:
    ts = infile.readlines()

In [ ]:
tss = [x.strip('\n').split('\t')[0:3] for x in ts if len(x.split('\t'))>1]

In [ ]:
ts_d = [{'speaker':x[0], 'start':x[1], 'end':x[2]} for x in tss]


In [ ]:
song = AudioSegment.from_file("Audios/Biden/Biden_1.mp3", 
                              ) 

In [ ]:
for i,limit in enumerate(ts_d):
    if limit['start'] == '0':
        limit['start'] = '0:0:0'
    clip = song[time2msec(limit['start']):time2msec(limit['end'])]
    clip.export(f"{limit['speaker']}_{i}.mp3", format='mp3')
    